# Paso 1: Unificación de los archivos CSV

En este proyecto tomaremos los archivos descargados de SISAIRE para Facatativá y los uniremos en un solo DataFrame.  
Cada CSV contiene mediciones de un contaminante diferente, por lo que el objetivo es obtener una única tabla donde:

- Cada fila corresponde a una fecha.
- Cada columna representa un contaminante.
- Todas las fechas coincidan entre sí.
- Se puedan visualizar juntos NO2, CO, PM2.5, PM10 y SO2.

Este paso es fundamental para el análisis de calidad del aire.

In [7]:
import pandas as pd

# Cargar archivos CSV
df1 = pd.read_csv("reporte_sisaire.csv")
df2 = pd.read_csv("reporte_sisaire (1).csv")
df3 = pd.read_csv("reporte_sisaire (2).csv")
df4 = pd.read_csv("reporte_sisaire (3).csv")
df5 = pd.read_csv("reporte_sisaire (4).csv")

# Unir todos los CSV
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

# Renombrar la columna para que todas tengan la misma 'fecha'
df = df.rename(columns={"Fecha inicial": "fecha"})

# Convertir la fecha a datetime
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

# Ordenar por fecha
df = df.sort_values("fecha")

# Una fila por día (la primera medición encontrada)
df_unido = df.groupby("fecha").first().reset_index()

df_unido.head()


,fecha,Estacion,Fecha final,NO2,CO,PM2.5,PM10,SO2
0,2023-05-08,FACATATIVA - URBANO,NaN,NaN,NaN,14.004167,32.972917,NaN
1,2023-05-09,FACATATIVA - URBANO,NaN,NaN,NaN,8.130833,21.382917,NaN
2,2023-05-10,FACATATIVA - URBANO,NaN,NaN,NaN,12.940417,28.497826,NaN
3,2023-05-11,FACATATIVA - URBANO,NaN,NaN,NaN,8.949167,22.261250,NaN
4,2023-05-12,FACATATIVA - URBANO,NaN,NaN,NaN,10.669167,27.921250,NaN


In [13]:
# Contar cuántos contaminantes NO son NaN por día
df_unido["datos_disponibles"] = df_unido.drop(columns=["fecha", "Estacion", "Fecha final"]).notna().sum(axis=1)

# Ordenar por cantidad de datos
df_top = df_unido.sort_values("datos_disponibles", ascending=False)

# Tomar los 10 días con más datos
top10 = df_top.head(10)

top10[["fecha", "datos_disponibles"]]


,fecha,datos_disponibles
895,2025-10-22,5
894,2025-10-21,5
893,2025-10-20,5
892,2025-10-19,5
891,2025-10-18,5
890,2025-10-17,5
889,2025-10-16,5
888,2025-10-15,5
887,2025-10-14,5
886,2025-10-13,5


In [14]:
# Obtener los 10 días con más datos
top10 = df_top.head(10)

# Filtrar el dataframe original para solo esos días
df_top10_completo = df_unido[df_unido["fecha"].isin(top10["fecha"])]

df_top10_completo


,fecha,Estacion,Fecha final,NO2,CO,PM2.5,PM10,SO2,datos_disponibles
886,2025-10-13,FACATATIVA - URBANO,NaN,20.066667,844.437500,6.168333,11.702917,8.329167,5
887,2025-10-14,FACATATIVA - URBANO,NaN,22.080417,1240.243182,12.690417,26.457083,8.060833,5
888,2025-10-15,FACATATIVA - URBANO,NaN,21.994167,1401.193750,16.507500,35.770000,8.687500,5
889,2025-10-16,FACATATIVA - URBANO,NaN,24.342083,1278.106250,12.292917,28.073750,8.329583,5
890,2025-10-17,FACATATIVA - URBANO,NaN,22.019167,1391.175000,16.489167,33.991667,9.327500,5
891,2025-10-18,FACATATIVA - URBANO,NaN,20.530000,1304.822917,9.441250,20.352917,8.300000,5
892,2025-10-19,FACATATIVA - URBANO,NaN,14.344167,1280.014583,10.534167,21.155833,8.406667,5
893,2025-10-20,FACATATIVA - URBANO,NaN,20.918750,991.379167,14.390833,31.004167,8.033913,5
894,2025-10-21,FACATATIVA - URBANO,NaN,22.931250,1015.710417,17.522500,34.707500,8.382083,5
895,2025-10-22,FACATATIVA - URBANO,NaN,23.710417,735.185417,13.423750,30.513333,8.341667,5


In [16]:
df_200 = df_unido.head(200)
df_200


,fecha,Estacion,Fecha final,NO2,CO,PM2.5,PM10,SO2,datos_disponibles
0,2023-05-08,FACATATIVA - URBANO,NaN,NaN,NaN,14.004167,32.972917,NaN,2
1,2023-05-09,FACATATIVA - URBANO,NaN,NaN,NaN,8.130833,21.382917,NaN,2
2,2023-05-10,FACATATIVA - URBANO,NaN,NaN,NaN,12.940417,28.497826,NaN,2
3,2023-05-11,FACATATIVA - URBANO,NaN,NaN,NaN,8.949167,22.261250,NaN,2
4,2023-05-12,FACATATIVA - URBANO,NaN,NaN,NaN,10.669167,27.921250,NaN,2
...,...,...,...,...,...,...,...,...,...
195,2023-11-22,FACATATIVA - URBANO,NaN,30.226957,962.754167,10.090833,20.101667,5.904545,5
196,2023-11-23,FACATATIVA - URBANO,NaN,34.216667,550.640909,14.433750,28.184583,3.123478,5
197,2023-11-24,FACATATIVA - URBANO,NaN,34.289167,513.341667,9.342500,20.308333,2.810000,5
198,2023-11-25,FACATATIVA - URBANO,NaN,29.537647,221.073077,12.285833,24.991250,3.025294,5


In [18]:
pd.set_option('display.max_rows', 250)   # muestra hasta 250 filas
pd.set_option('display.max_columns', 20) # muestra todas las columnas

df_200


,fecha,Estacion,Fecha final,NO2,CO,PM2.5,PM10,SO2,datos_disponibles
0,2023-05-08,FACATATIVA - URBANO,NaN,NaN,NaN,14.004167,32.972917,NaN,2
1,2023-05-09,FACATATIVA - URBANO,NaN,NaN,NaN,8.130833,21.382917,NaN,2
2,2023-05-10,FACATATIVA - URBANO,NaN,NaN,NaN,12.940417,28.497826,NaN,2
3,2023-05-11,FACATATIVA - URBANO,NaN,NaN,NaN,8.949167,22.261250,NaN,2
4,2023-05-12,FACATATIVA - URBANO,NaN,NaN,NaN,10.669167,27.921250,NaN,2
5,2023-05-13,FACATATIVA - URBANO,NaN,NaN,NaN,10.308333,22.472917,NaN,2
6,2023-05-14,FACATATIVA - URBANO,NaN,NaN,NaN,4.887083,10.618750,NaN,2
7,2023-05-15,FACATATIVA - URBANO,NaN,NaN,NaN,7.746667,21.407500,NaN,2
8,2023-05-16,FACATATIVA - URBANO,NaN,NaN,NaN,12.740000,25.460435,NaN,2
9,2023-05-17,FACATATIVA - URBANO,NaN,NaN,NaN,11.430000,23.952500,NaN,2


In [19]:
import os

# Crear carpeta de salida
os.makedirs("exportado", exist_ok=True)

# Exportar todos los dataframes existentes
df_unido.to_csv("exportado/df_unido_completo.csv", index=False)
df_top.to_csv("exportado/df_dias_ordenados_por_datos.csv", index=False)
top10.to_csv("exportado/top10_dias.csv", index=False)
df_200.to_csv("exportado/df_200_dias.csv", index=False)

print("Listo. Los CSV se guardaron en la carpeta 'exportado'.")


Listo. Los CSV se guardaron en la carpeta 'exportado'.
